### Bag of words

In [1]:
# 문서를 단어들의 집합으로 나타내는 방법
# 문서에 등장하는 단어의 빈도만 집중, 단어의 순서나 문맥을 고려하지는 않음

In [2]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 50.8 MB/s eta 0:00:00


In [3]:
from konlpy.tag import Okt

okt = Okt()
text = '나는 오늘 파이썬을 공부하고 있습니다. 파이썬은 정말 재미있는 언어입니다.'

In [5]:
document = text.replace('.','')
tokenized_document = okt.morphs(document)
tokenized_document

['나',
 '는',
 '오늘',
 '파이썬',
 '을',
 '공부',
 '하고',
 '있습니다',
 '파이썬',
 '은',
 '정말',
 '재미있는',
 '언어',
 '입니다']

In [9]:
word_to_index = {}
bow = []

for word in tokenized_document:
  # 처음 등
  if word not in word_to_index.keys():
    word_to_index[word] = len(word_to_index)
    bow.insert(len(word_to_index)-1,1)

  # 재등장
  else:
    index = word_to_index.get(word)
    bow[index] += 1

  print(word_to_index)
  print(bow)

{'나': 0}
[1]
{'나': 0, '는': 1}
[1, 1]
{'나': 0, '는': 1, '오늘': 2}
[1, 1, 1]
{'나': 0, '는': 1, '오늘': 2, '파이썬': 3}
[1, 1, 1, 1]
{'나': 0, '는': 1, '오늘': 2, '파이썬': 3, '을': 4}
[1, 1, 1, 1, 1]
{'나': 0, '는': 1, '오늘': 2, '파이썬': 3, '을': 4, '공부': 5}
[1, 1, 1, 1, 1, 1]
{'나': 0, '는': 1, '오늘': 2, '파이썬': 3, '을': 4, '공부': 5, '하고': 6}
[1, 1, 1, 1, 1, 1, 1]
{'나': 0, '는': 1, '오늘': 2, '파이썬': 3, '을': 4, '공부': 5, '하고': 6, '있습니다': 7}
[1, 1, 1, 1, 1, 1, 1, 1]
{'나': 0, '는': 1, '오늘': 2, '파이썬': 3, '을': 4, '공부': 5, '하고': 6, '있습니다': 7}
[1, 1, 1, 2, 1, 1, 1, 1]
{'나': 0, '는': 1, '오늘': 2, '파이썬': 3, '을': 4, '공부': 5, '하고': 6, '있습니다': 7, '은': 8}
[1, 1, 1, 2, 1, 1, 1, 1, 1]
{'나': 0, '는': 1, '오늘': 2, '파이썬': 3, '을': 4, '공부': 5, '하고': 6, '있습니다': 7, '은': 8, '정말': 9}
[1, 1, 1, 2, 1, 1, 1, 1, 1, 1]
{'나': 0, '는': 1, '오늘': 2, '파이썬': 3, '을': 4, '공부': 5, '하고': 6, '있습니다': 7, '은': 8, '정말': 9, '재미있는': 10}
[1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1]
{'나': 0, '는': 1, '오늘': 2, '파이썬': 3, '을': 4, '공부': 5, '하고': 6, '있습니다': 7, '은': 8, '정말': 9, '재미있는': 10

### DTM

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

text = [
  ['누구나','한번쯤','사랑','웃고'],
  ['누구나','한번쯤','사랑','웃고'],
  ['그것','바로','사랑','사랑','사랑']
]

In [11]:
count_vec = CountVectorizer(tokenizer=lambda x:x,lowercase=False)
print(count_vec)
result = count_vec.fit_transform(text)
print(count_vec.vocabulary_)

CountVectorizer(lowercase=False,
                tokenizer=<function <lambda> at 0x7f724119d870>)
{'누구나': 1, '한번쯤': 5, '사랑': 3, '웃고': 4, '그것': 0, '바로': 2}


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [13]:
result.toarray()

array([[0, 1, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 1],
       [1, 0, 1, 3, 0, 0]])

### TF-IDF

In [14]:
# 문서 내에서 특정 단어가 얼마나 중요한지 나타내는 가중치
# bow에서 빈도수에만 기반하여 가중치를 주는 방식을개선

In [15]:
import pandas as pd
from math import log
okt = Okt()

In [16]:
data = '''
나는 영어를 좋아합니다.
영어,수학을 매일 공부합니다.
과학 공부를 좋아하고 수학도 좋아합니다.
영어,수학,과학을 좋아 하고 영어는 매일 공부합니다.
'''

In [20]:
words = okt.morphs(data)
result = []
for word in words:
  # 한글자 이상인 단어만 저장
  if len(word) > 1:
    result.append(word)

# 중복되는 단어들이 많이 나옴
result

['영어',
 '좋아합니다',
 '영어',
 '수학',
 '매일',
 '공부',
 '합니다',
 '과학',
 '공부',
 '좋아하고',
 '수학',
 '좋아합니다',
 '영어',
 '수학',
 '과학',
 '좋아',
 '하고',
 '영어',
 '매일',
 '공부',
 '합니다']

In [21]:
result = list(set(result))
result

['영어', '공부', '매일', '좋아합니다', '수학', '하고', '좋아', '합니다', '좋아하고', '과학']

In [28]:
documents = data.split('.')
documents

['\n나는 영어를 좋아합니다',
 '\n영어,수학을 매일 공부합니다',
 '\n과학 공부를 좋아하고 수학도 좋아합니다',
 '\n영어,수학,과학을 좋아 하고 영어는 매일 공부합니다',
 '\n']

In [30]:
d_list = []
for tmp in documents:
  # 공백 제거
  if tmp.strip() != '':
    d_list.append(tmp.strip())

d_list

['나는 영어를 좋아합니다',
 '영어,수학을 매일 공부합니다',
 '과학 공부를 좋아하고 수학도 좋아합니다',
 '영어,수학,과학을 좋아 하고 영어는 매일 공부합니다']

In [42]:
N = len(d_list)
def tf(t,d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in d_list:
        df += t in doc
    return log(N/(df+1))

def tfidf(t,d):
    return tf(t,d) * idf(t)

In [33]:
word_list = result
t_result = []

for tmp in range(N):
  t_result.append([])
  d = d_list[tmp]
  print("d_list : ",d_list)
  print("d : ",d)
  for tmp1 in range(len(word_list)):
    t = word_list[tmp1]
    t_result[-1].append(tf(t,d))
  print("t : ",t)

d, t_result

d_list :  ['나는 영어를 좋아합니다', '영어,수학을 매일 공부합니다', '과학 공부를 좋아하고 수학도 좋아합니다', '영어,수학,과학을 좋아 하고 영어는 매일 공부합니다']
d :  나는 영어를 좋아합니다
t :  과학
d_list :  ['나는 영어를 좋아합니다', '영어,수학을 매일 공부합니다', '과학 공부를 좋아하고 수학도 좋아합니다', '영어,수학,과학을 좋아 하고 영어는 매일 공부합니다']
d :  영어,수학을 매일 공부합니다
t :  과학
d_list :  ['나는 영어를 좋아합니다', '영어,수학을 매일 공부합니다', '과학 공부를 좋아하고 수학도 좋아합니다', '영어,수학,과학을 좋아 하고 영어는 매일 공부합니다']
d :  과학 공부를 좋아하고 수학도 좋아합니다
t :  과학
d_list :  ['나는 영어를 좋아합니다', '영어,수학을 매일 공부합니다', '과학 공부를 좋아하고 수학도 좋아합니다', '영어,수학,과학을 좋아 하고 영어는 매일 공부합니다']
d :  영어,수학,과학을 좋아 하고 영어는 매일 공부합니다
t :  과학


('영어,수학,과학을 좋아 하고 영어는 매일 공부합니다',
 [[1, 0, 0, 1, 0, 0, 1, 1, 0, 0],
  [1, 1, 1, 0, 1, 0, 0, 1, 0, 0],
  [0, 1, 0, 1, 1, 1, 2, 1, 1, 1],
  [2, 1, 1, 0, 1, 1, 1, 1, 0, 1]])

In [38]:
df = pd.DataFrame(t_result , columns=word_list)
df

,영어,공부,매일,좋아합니다,수학,하고,좋아,합니다,좋아하고,과학
0,1,0,0,1,0,0,1,1,0,0
1,1,1,1,0,1,0,0,1,0,0
2,0,1,0,1,1,1,2,1,1,1
3,2,1,1,0,1,1,1,1,0,1


In [43]:
tfidf_result = []
for temp in range(len(word_list)):
    t = word_list[temp]
    tfidf_result.append(idf(t))

print(tfidf_result)

[0.0, 0.0, 0.28768207245178085, 0.28768207245178085, 0.0, 0.28768207245178085, 0.0, -0.2231435513142097, 0.6931471805599453, 0.28768207245178085]


In [44]:
df_idf = pd.DataFrame(tfidf_result, index=word_list, columns=['IDF'])
df_idf

,IDF
영어,0.000000
공부,0.000000
매일,0.287682
좋아합니다,0.287682
수학,0.000000
하고,0.287682
좋아,0.000000
합니다,-0.223144
좋아하고,0.693147
과학,0.287682


In [47]:
result = []

for temp in range(N):
    result.append([])
    d = d_list[temp]
    for temp in range(len(word_list)):
        t = word_list[temp]
        result[-1].append(tfidf(t,d))

tf_idf_df = pd.DataFrame(result,columns=word_list)
tf_idf_df

,영어,공부,매일,좋아합니다,수학,하고,좋아,합니다,좋아하고,과학
0,0.0,0.0,0.000000,0.287682,0.0,0.000000,0.0,-0.223144,0.000000,0.000000
1,0.0,0.0,0.287682,0.000000,0.0,0.000000,0.0,-0.223144,0.000000,0.000000
2,0.0,0.0,0.000000,0.287682,0.0,0.287682,0.0,-0.223144,0.693147,0.287682
3,0.0,0.0,0.287682,0.000000,0.0,0.287682,0.0,-0.223144,0.000000,0.287682
